In [5]:
import requests
import base64
import pandas as pd
import json
import csv
import glob
import os
import chart_studio.plotly as py
import plotly.graph_objects as go


In [15]:
prov_abbs = ['AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'QC', 'ON', 'SK', 'PE', 'NT', 'NU', 'YT']
url="https://api.covid19tracker.ca/reports/province/"

def access_api(prov_list, url):
    for prov in prov_list:
        new_url = url + prov
        r = requests.get(new_url, allow_redirects=True)

        open('data_summary.json', 'wb').write(r.content)

        with open('data_summary.json') as json_file:
            data = json.load(json_file)

        hosp = data['data']
        
        filename = prov + 'data_file.csv'
        data_file = open(filename, 'w')

        csv_writer = csv.writer(data_file)

        count = 0
        for person in hosp:
            if count == 0:
                header = person.keys()
                csv_writer.writerow(header)
                count += 1

            csv_writer.writerow(person.values())

        data_file.close()



In [16]:
access_api(prov_abbs, url)



In [18]:
prov_dict = {
    'AB':'Alberta', 'BC':'British Columbia', 'MB':'Manitoba', 'NB':'New Brunswick', 'NL':'Newfoundland', 'NS':'Nova Scotia', 'QC':'Quebec', 
    'ON':'Ontario', 'SK':'Saskatchewan', 'PE':'Prince Edward Island', 'NT':'Northwest Territories', 'NU':'Nunavut', 'YT':'Yukon'
}

In [25]:
def make_prov_dataset(file_string, column):
    column_names = ['date', column, 'prname']
    data = pd.DataFrame(columns = column_names)
    for filename in glob.glob(file_string):
        with open(os.path.join(os.getcwd(), filename), 'r') as f:
            df = pd.read_csv(f)
            df = df[['date', column]]
            df['prname'] = filename[:2]
            data = data.append(df)

    return data

pr_data = make_prov_dataset('*data_file.csv', 'total_hospitalizations')

         date  total_hospitalizations prname
0  2020-01-25                       0     QC
1  2020-01-26                       0     QC
2  2020-01-27                       0     QC
3  2020-01-28                       0     QC
4  2020-01-29                       0     QC
         date total_hospitalizations prname
0  2020-01-25                      0     QC
1  2020-01-26                      0     QC
2  2020-01-27                      0     QC
3  2020-01-28                      0     QC
4  2020-01-29                      0     QC
         date  total_hospitalizations prname
0  2020-01-25                       0     YT
1  2020-01-26                       0     YT
2  2020-01-27                       0     YT
3  2020-01-28                       0     YT
4  2020-01-29                       0     YT
         date total_hospitalizations prname
0  2020-01-25                      0     QC
1  2020-01-26                      0     QC
2  2020-01-27                      0     QC
3  2020-01-28       

In [27]:
pr_data['date'] = pd.to_datetime(pr_data['date'])


In [30]:
graph_two = []
for region in prov_abbs:
        x_val = pr_data[pr_data['prname'] == region].date.tolist()
        y_val = pr_data[pr_data['prname'] == region].total_hospitalizations.tolist()
        graph_two.append(
            go.Scatter(
                x = x_val,
                y = y_val,
                mode = 'lines',
                name = region
            )
        )

layout_two = dict(title = 'Cases Active Historically By Region',
                        xaxis = dict(title = 'Date'),
                        yaxis = dict(title = 'Active Cases'),
                    )

fig = go.Figure(data=graph_two, layout=layout_two)

#py.iplot(fig, sharing='private', filename='jupyter-styled_bar')
print(graph_two)

ValueError: 
    Invalid value of type 'builtins.str' received for the 'mode' property of scatter
        Received value: 'line'

    The 'mode' property is a flaglist and may be specified
    as a string containing:
      - Any combination of ['lines', 'markers', 'text'] joined with '+' characters
        (e.g. 'lines+markers')
        OR exactly one of ['none'] (e.g. 'none')